In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading builders for datasets

In [2]:
RECIPES5K_DIR = "./Food Datasets/final-dataset/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "./Food Datasets/final-dataset/tfrecord/food101/1.1.0"
NUTRITION5K_DIR = "./Food Datasets/final-dataset/tfrecord/nutrition5k/1.0.0"

# Building Dataset Loaders
# Creates a TensorFlow dataset builder from the specified directories for Recipes5k and Food101 datasets.
recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
#Creates a TensorFlow dataset builder from the specified directory for the Nutrition5k dataset.
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

Retrieves the number of examples in the training split of the Recipes5k, Food101, and Nutrition5k datasets.

In [3]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [4]:
nutrition5k_builder.info.splits["train"].num_examples

271829

In [5]:
# Converts category and ingredients from tensors to strings, then uses the one-hot encoder to encode them.
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [6]:
# Prepares the dataset for training by applying the encoding function and formatting the output.
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training ingredients and nutrients only

This code snippet is focused on preparing the Nutrition5k dataset for training a model that predicts ingredients and nutritional values (calories, carbs, protein, fat). The dataset is split into training and validation sets, with appropriate preprocessing steps applied, including encoding categorical features and batching the data for efficient training. The datasets are also prefetched to improve performance during model training.

In [7]:
# FULL DATASET
no_category_train, no_category_val = nutrition5k_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"]
)

In [8]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")

Total training size : 217463
Total validation size : 54366


In [9]:
# Prefetches the batches to optimize training performance by overlapping the data loading with the model training. tf.data.AUTOTUNE automatically adjusts the prefetch buffer size.
BATCH_SIZE = 256
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training category, ingredients and nutrients

This code snippet is focused on preparing datasets for training a model that predicts food categories, ingredients, and nutritional values (calories, carbs, protein, fat). The datasets from Recipes5k, Food101, and a subset of Nutrition5k are combined and split into training and validation sets. The datasets undergo preprocessing steps, including encoding categorical features and batching the data for efficient training. Prefetching is applied to improve performance during model training.

In [10]:
# FULL DATASET
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
) = recipes5k_food101_builder.as_dataset(split=["train[:80%]", "train[80%:]"])

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(nutrition5k_category_train, nutrition5k_category_val) = nutrition5k_builder.as_dataset(
    split=["train[:800]", "train[800:1000]"]
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)

In [11]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

Total training size : 85461
Total validation size : 21365


In [12]:
BATCH_SIZE = 256
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

# Model Development

BaseModel class that serves as a blueprint for creating a complex neural network model for food classification and nutrient regression tasks

This BaseModel class serves as a foundation for constructing a complex neural network model for food classification and nutritional value prediction. It defines methods for building, compiling, training, and utilizing the model, as well as managing its layers and callbacks effectively. This modular approach facilitates the construction of sophisticated deep learning models tailored to specific tasks in food analysis.

In [13]:
class BaseModel:
    # Sets up basic attributes such as input_shape, total categories for food and ingredients, model name, and configuration name.
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_units,
    ):
        raise NotImplementedError

    def compile_model(self, optimizer):
        assert self.model is not None
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        self.model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )

    def freeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def freeze_convolution_base(self):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetB1")
        submodel.trainable = False

    def get_callbacks(self):
        # Define callbacks for training
        tensorboard_dir = f"./models/logs/{self.name}/{self.model_config_name}"
        checkpoint_dir = f"./temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_model_path = checkpoint_dir + "/model"
        checkpoint_weights_path = checkpoint_dir + "/weights/ckpt"
        # ModelCheckpoint to save best model
        checkpoint_model_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_model_path, monitor="val_loss", save_best_only=True, mode="min"
        )
        # ModelCheckpoint to save best weights
        checkpoint_weights_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_weights_path,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            mode="min",
        )
        # Reduce learning rate on plateau
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        # TensorBoard for visualization
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        # Early stopping to prevent overfitting
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=8)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_model_callback,
            checkpoint_weights_callback,
            reduce_lr_callback,
        ]

    # Layers from bottom to top (classifier)  #
    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
        return output_model

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    # End of layers  #

    def load_model(self, path):
        self.model = keras.models.load_model(path)

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        self.model.summary(expand_nested=True, show_trainable=True)

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert self.model is not None, "No model found."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetb1")
        submodel.trainable = True
        for layer in submodel.layers[:fine_tune_at]:
            layer.trainable = False

## Flat Model

The FlatModel class inherits from the BaseModel class and implements the model building process, specifying the architecture of the neural network.

In [14]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

### EfficientNetB1 (Best)

The FlatEfficientNetB1Model class extends the FlatModel class, using the EfficientNetB1 architecture for the convolutional base.

The FlatEfficientNetB1Model class leverages the EfficientNetB1 architecture as its convolutional base, providing a powerful and efficient feature extractor pretrained on ImageNet. It includes preprocessing specific to EfficientNet and uses a frozen convolutional base to maintain the pretrained weights. This class is designed for tasks requiring high-quality feature extraction, such as food classification and nutrient estimation, making it suitable for the dataset and problem at hand.

In [ ]:
class FlatEfficientNetB1Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on ingredients dataset (nutrition5k)

This code sets up and trains an EfficientNetB1-based model to predict multiple outputs (ingredients, protein, fat, carbs, and calories) from images. The model is built with specific layer configurations, compiled with an optimizer, and trained on a dataset with appropriate callbacks and logging. This approach leverages transfer learning by freezing the pretrained convolutional base and focusing on training the new dense layers added on top.

In [16]:
# This section of the code trains the FlatEfficientNetB1Model on the ingredients dataset (nutrition5k), focusing on building and compiling the model, and then training it.
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalIngredients",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
# Freezing Category Layers: Freezes the layers related to category classification to prevent them from being trained in this stage.
flat_efficientnet.freeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 augmentation_layers (Functiona  (None, 224, 224, 3)  0          ['input_1[0][0]']                Y          
 l)                                                                                                          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_2 (InputLayer)         [(None, 224, 224, 3  0           []                         

In [17]:
flat_efficientnet.train_model(
    x=no_category_train,
    validation_data=no_category_val,
    epochs=5,
    # Verbose: Sets verbosity to 2, providing detailed logs for each epoch.
    verbose=2,
)

Epoch 1/5
INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalIngredients\model\assets


INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalIngredients\model\assets


850/850 - 1645s - loss: 0.9321 - category_output_loss: 0.0725 - ingredients_output_loss: 0.0655 - calorie_output_loss: 0.5436 - carbs_output_loss: 0.0864 - protein_output_loss: 0.0767 - fat_output_loss: 0.0875 - category_output_acc: 0.9892 - category_output_precision: 1.0000 - category_output_recall: 0.9814 - ingredients_output_precision: 0.0719 - ingredients_output_recall: 0.2803 - calorie_output_MAE: 0.5436 - carbs_output_MAE: 0.0864 - protein_output_MAE: 0.0767 - fat_output_MAE: 0.0875 - val_loss: 0.5431 - val_category_output_loss: 0.0020 - val_ingredients_output_loss: 0.0081 - val_calorie_output_loss: 0.3982 - val_carbs_output_loss: 0.0494 - val_protein_output_loss: 0.0365 - val_fat_output_loss: 0.0489 - val_category_output_acc: 1.0000 - val_category_output_precision: 1.0000 - val_category_output_recall: 1.0000 - val_ingredients_output_precision: 0.8191 - val_ingredients_output_recall: 0.4131 - val_calorie_output_MAE: 0.3982 - val_carbs_output_MAE: 0.0494 - val_protein_output_MAE: 

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalIngredients\model\assets


850/850 - 1621s - loss: 0.5035 - category_output_loss: 0.0033 - ingredients_output_loss: 0.0074 - calorie_output_loss: 0.3659 - carbs_output_loss: 0.0480 - protein_output_loss: 0.0356 - fat_output_loss: 0.0433 - category_output_acc: 1.0000 - category_output_precision: 1.0000 - category_output_recall: 1.0000 - ingredients_output_precision: 0.8071 - ingredients_output_recall: 0.4583 - calorie_output_MAE: 0.3659 - carbs_output_MAE: 0.0480 - protein_output_MAE: 0.0356 - fat_output_MAE: 0.0433 - val_loss: 0.4228 - val_category_output_loss: 0.0017 - val_ingredients_output_loss: 0.0064 - val_calorie_output_loss: 0.3081 - val_carbs_output_loss: 0.0419 - val_protein_output_loss: 0.0299 - val_fat_output_loss: 0.0348 - val_category_output_acc: 1.0000 - val_category_output_precision: 1.0000 - val_category_output_recall: 1.0000 - val_ingredients_output_precision: 0.7709 - val_ingredients_output_recall: 0.5798 - val_calorie_output_MAE: 0.3081 - val_carbs_output_MAE: 0.0419 - val_protein_output_MAE: 

...

850/850 - 1624s - loss: 0.4438 - category_output_loss: 0.0023 - ingredients_output_loss: 0.0062 - calorie_output_loss: 0.3260 - carbs_output_loss: 0.0427 - protein_output_loss: 0.0302 - fat_output_loss: 0.0365 - category_output_acc: 1.0000 - category_output_precision: 1.0000 - category_output_recall: 1.0000 - ingredients_output_precision: 0.8216 - ingredients_output_recall: 0.5381 - calorie_output_MAE: 0.3260 - carbs_output_MAE: 0.0427 - protein_output_MAE: 0.0302 - fat_output_MAE: 0.0365 - val_loss: 0.3786 - val_category_output_loss: 0.0012 - val_ingredients_output_loss: 0.0055 - val_calorie_output_loss: 0.2772 - val_carbs_output_loss: 0.0365 - val_protein_output_loss: 0.0271 - val_fat_output_loss: 0.0310 - val_category_output_acc: 1.0000 - val_category_output_precision: 1.0000 - val_category_output_recall: 1.0000 - val_ingredients_output_precision: 0.8435 - val_ingredients_output_recall: 0.5859 - val_calorie_output_MAE: 0.2772 - val_carbs_output_MAE: 0.0365 - val_protein_output_MAE: 

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalIngredients\model\assets


850/850 - 1619s - loss: 0.4077 - category_output_loss: 0.0017 - ingredients_output_loss: 0.0055 - calorie_output_loss: 0.3016 - carbs_output_loss: 0.0393 - protein_output_loss: 0.0270 - fat_output_loss: 0.0325 - category_output_acc: 1.0000 - category_output_precision: 1.0000 - category_output_recall: 1.0000 - ingredients_output_precision: 0.8331 - ingredients_output_recall: 0.5923 - calorie_output_MAE: 0.3016 - carbs_output_MAE: 0.0393 - protein_output_MAE: 0.0270 - fat_output_MAE: 0.0325 - val_loss: 0.3557 - val_category_output_loss: 9.4994e-04 - val_ingredients_output_loss: 0.0049 - val_calorie_output_loss: 0.2620 - val_carbs_output_loss: 0.0350 - val_protein_output_loss: 0.0256 - val_fat_output_loss: 0.0272 - val_category_output_acc: 1.0000 - val_category_output_precision: 1.0000 - val_category_output_recall: 0.9999 - val_ingredients_output_precision: 0.8414 - val_ingredients_output_recall: 0.6491 - val_calorie_output_MAE: 0.2620 - val_carbs_output_MAE: 0.0350 - val_protein_output_M

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalIngredients\model\assets


850/850 - 1642s - loss: 0.3825 - category_output_loss: 0.0013 - ingredients_output_loss: 0.0050 - calorie_output_loss: 0.2846 - carbs_output_loss: 0.0368 - protein_output_loss: 0.0247 - fat_output_loss: 0.0301 - category_output_acc: 1.0000 - category_output_precision: 1.0000 - category_output_recall: 1.0000 - ingredients_output_precision: 0.8425 - ingredients_output_recall: 0.6328 - calorie_output_MAE: 0.2846 - carbs_output_MAE: 0.0368 - protein_output_MAE: 0.0247 - fat_output_MAE: 0.0301 - val_loss: 0.3465 - val_category_output_loss: 0.0022 - val_ingredients_output_loss: 0.0044 - val_calorie_output_loss: 0.2519 - val_carbs_output_loss: 0.0358 - val_protein_output_loss: 0.0270 - val_fat_output_loss: 0.0251 - val_category_output_acc: 0.9998 - val_category_output_precision: 0.9999 - val_category_output_recall: 0.9996 - val_ingredients_output_precision: 0.8421 - val_ingredients_output_recall: 0.6961 - val_calorie_output_MAE: 0.2519 - val_carbs_output_MAE: 0.0358 - val_protein_output_MAE: 

#### Train on main dataset (recipes5k + food101 + nutrition5k) with weights transfer from ingredients dataset

This code continues the training process by first initializing a new FlatEfficientNetB1Model instance, then loading the pretrained weights from the model trained on the ingredients dataset. It unfreezes the category classification layers to allow further training on the comprehensive dataset, compiles the model with an Adam optimizer, and prints the model summary. Finally, it trains the model on the combined dataset, which includes recipes, food images, and nutritional information, aiming to leverage the pretrained weights for improved performance.

In [18]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategory",
)

flat_efficientnet.load_model("./temp/checkpoint/flat_efficientnetB1/finalIngredients/model")
# Unfreezes the category classification layers to allow them to be trained on the new dataset.
flat_efficientnet.unfreeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 augmentation_layers (Functiona  (None, 224, 224, 3)  0          ['input_1[0][0]']                Y          
 l)                                                                                                          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_2 (InputLayer)         [(None, 224, 224, 3  0           []                         

In [19]:
flat_efficientnet.train_model(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=5,
    verbose=2,
)

Epoch 1/5
INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategory\model\assets


INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategory\model\assets


334/334 - 720s - loss: 3.9763 - category_output_loss: 3.1400 - ingredients_output_loss: 0.0362 - calorie_output_loss: 0.6276 - carbs_output_loss: 0.0791 - protein_output_loss: 0.0299 - fat_output_loss: 0.0635 - category_output_acc: 0.3329 - category_output_precision: 0.5976 - category_output_recall: 0.1608 - ingredients_output_precision: 0.6586 - ingredients_output_recall: 0.1634 - calorie_output_MAE: 0.6276 - carbs_output_MAE: 0.0791 - protein_output_MAE: 0.0299 - fat_output_MAE: 0.0635 - val_loss: 2.5892 - val_category_output_loss: 1.8525 - val_ingredients_output_loss: 0.0263 - val_calorie_output_loss: 0.5407 - val_carbs_output_loss: 0.0718 - val_protein_output_loss: 0.0341 - val_fat_output_loss: 0.0638 - val_category_output_acc: 0.5428 - val_category_output_precision: 0.8153 - val_category_output_recall: 0.3712 - val_ingredients_output_precision: 0.6284 - val_ingredients_output_recall: 0.3659 - val_calorie_output_MAE: 0.5407 - val_carbs_output_MAE: 0.0718 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategory\model\assets


334/334 - 690s - loss: 2.7614 - category_output_loss: 2.0474 - ingredients_output_loss: 0.0245 - calorie_output_loss: 0.5391 - carbs_output_loss: 0.0704 - protein_output_loss: 0.0234 - fat_output_loss: 0.0566 - category_output_acc: 0.5046 - category_output_precision: 0.8158 - category_output_recall: 0.3166 - ingredients_output_precision: 0.7293 - ingredients_output_recall: 0.3130 - calorie_output_MAE: 0.5391 - carbs_output_MAE: 0.0704 - protein_output_MAE: 0.0234 - fat_output_MAE: 0.0566 - val_loss: 2.3296 - val_category_output_loss: 1.6195 - val_ingredients_output_loss: 0.0223 - val_calorie_output_loss: 0.5230 - val_carbs_output_loss: 0.0705 - val_protein_output_loss: 0.0329 - val_fat_output_loss: 0.0614 - val_category_output_acc: 0.5944 - val_category_output_precision: 0.8244 - val_category_output_recall: 0.4398 - val_ingredients_output_precision: 0.7862 - val_ingredients_output_recall: 0.3796 - val_calorie_output_MAE: 0.5230 - val_carbs_output_MAE: 0.0705 - val_protein_output_MAE: 0

...

334/334 - 706s - loss: 2.5205 - category_output_loss: 1.8287 - ingredients_output_loss: 0.0227 - calorie_output_loss: 0.5240 - carbs_output_loss: 0.0679 - protein_output_loss: 0.0222 - fat_output_loss: 0.0549 - category_output_acc: 0.5478 - category_output_precision: 0.8244 - category_output_recall: 0.3726 - ingredients_output_precision: 0.7588 - ingredients_output_recall: 0.3757 - calorie_output_MAE: 0.5240 - carbs_output_MAE: 0.0679 - protein_output_MAE: 0.0222 - fat_output_MAE: 0.0549 - val_loss: 2.2181 - val_category_output_loss: 1.5407 - val_ingredients_output_loss: 0.0209 - val_calorie_output_loss: 0.5023 - val_carbs_output_loss: 0.0670 - val_protein_output_loss: 0.0267 - val_fat_output_loss: 0.0605 - val_category_output_acc: 0.6125 - val_category_output_precision: 0.8238 - val_category_output_recall: 0.4724 - val_ingredients_output_precision: 0.7973 - val_ingredients_output_recall: 0.4394 - val_calorie_output_MAE: 0.5023 - val_carbs_output_MAE: 0.0670 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategory\model\assets


334/334 - 694s - loss: 2.3864 - category_output_loss: 1.7093 - ingredients_output_loss: 0.0217 - calorie_output_loss: 0.5138 - carbs_output_loss: 0.0661 - protein_output_loss: 0.0215 - fat_output_loss: 0.0539 - category_output_acc: 0.5727 - category_output_precision: 0.8287 - category_output_recall: 0.4070 - ingredients_output_precision: 0.7740 - ingredients_output_recall: 0.4138 - calorie_output_MAE: 0.5138 - carbs_output_MAE: 0.0661 - protein_output_MAE: 0.0215 - fat_output_MAE: 0.0539 - val_loss: 2.1409 - val_category_output_loss: 1.4832 - val_ingredients_output_loss: 0.0200 - val_calorie_output_loss: 0.4936 - val_carbs_output_loss: 0.0633 - val_protein_output_loss: 0.0258 - val_fat_output_loss: 0.0550 - val_category_output_acc: 0.6243 - val_category_output_precision: 0.8309 - val_category_output_recall: 0.4916 - val_ingredients_output_precision: 0.8123 - val_ingredients_output_recall: 0.4709 - val_calorie_output_MAE: 0.4936 - val_carbs_output_MAE: 0.0633 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategory\model\assets


334/334 - 706s - loss: 2.2828 - category_output_loss: 1.6160 - ingredients_output_loss: 0.0209 - calorie_output_loss: 0.5063 - carbs_output_loss: 0.0650 - protein_output_loss: 0.0213 - fat_output_loss: 0.0533 - category_output_acc: 0.5920 - category_output_precision: 0.8342 - category_output_recall: 0.4321 - ingredients_output_precision: 0.7868 - ingredients_output_recall: 0.4392 - calorie_output_MAE: 0.5063 - carbs_output_MAE: 0.0650 - protein_output_MAE: 0.0213 - fat_output_MAE: 0.0533 - val_loss: 2.1083 - val_category_output_loss: 1.4575 - val_ingredients_output_loss: 0.0194 - val_calorie_output_loss: 0.4870 - val_carbs_output_loss: 0.0651 - val_protein_output_loss: 0.0245 - val_fat_output_loss: 0.0547 - val_category_output_acc: 0.6309 - val_category_output_precision: 0.8312 - val_category_output_recall: 0.5061 - val_ingredients_output_precision: 0.8191 - val_ingredients_output_recall: 0.4926 - val_calorie_output_MAE: 0.4870 - val_carbs_output_MAE: 0.0651 - val_protein_output_MAE: 0

#### Train on main dataset (recipes5k + food101 + nutrition5k) without weights transfer

This code sets up a new FlatEfficientNetB1Model instance without loading any pretrained weights. The model is built from scratch with the specified shared and independent layer units. After compiling the model with the Adam optimizer, it prints the model summary and begins training on the comprehensive dataset. This approach allows the model to learn directly from the main dataset without any influence from previously trained weights, aiming to evaluate the impact of starting training from scratch versus transferring pretrained weights.

In [20]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 augmentation_layers (Functiona  (None, 224, 224, 3)  0          ['input_12[0][0]']               Y          
 l)                                                                                                          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_13 (InputLayer)        [(None, 224, 224, 3  0           []                         

In [21]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=50, verbose=2
)

Epoch 1/50
INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer\model\assets


INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer\model\assets


334/334 - 712s - loss: 3.8537 - category_output_loss: 2.2897 - ingredients_output_loss: 0.1601 - calorie_output_loss: 1.0036 - carbs_output_loss: 0.1538 - protein_output_loss: 0.1257 - fat_output_loss: 0.1208 - category_output_acc: 0.4424 - category_output_precision: 0.7460 - category_output_recall: 0.2919 - ingredients_output_precision: 0.0573 - ingredients_output_recall: 0.2648 - calorie_output_MAE: 1.0036 - carbs_output_MAE: 0.1538 - protein_output_MAE: 0.1257 - fat_output_MAE: 0.1208 - val_loss: 2.4061 - val_category_output_loss: 1.6442 - val_ingredients_output_loss: 0.0298 - val_calorie_output_loss: 0.5369 - val_carbs_output_loss: 0.0720 - val_protein_output_loss: 0.0526 - val_fat_output_loss: 0.0705 - val_category_output_acc: 0.5740 - val_category_output_precision: 0.8181 - val_category_output_recall: 0.4171 - val_ingredients_output_precision: 0.7840 - val_ingredients_output_recall: 0.2458 - val_calorie_output_MAE: 0.5369 - val_carbs_output_MAE: 0.0720 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer\model\assets


334/334 - 707s - loss: 2.5496 - category_output_loss: 1.8172 - ingredients_output_loss: 0.0246 - calorie_output_loss: 0.5389 - carbs_output_loss: 0.0765 - protein_output_loss: 0.0329 - fat_output_loss: 0.0595 - category_output_acc: 0.5398 - category_output_precision: 0.7877 - category_output_recall: 0.3952 - ingredients_output_precision: 0.7533 - ingredients_output_recall: 0.3356 - calorie_output_MAE: 0.5389 - carbs_output_MAE: 0.0765 - protein_output_MAE: 0.0329 - fat_output_MAE: 0.0595 - val_loss: 2.2201 - val_category_output_loss: 1.5216 - val_ingredients_output_loss: 0.0231 - val_calorie_output_loss: 0.5158 - val_carbs_output_loss: 0.0675 - val_protein_output_loss: 0.0327 - val_fat_output_loss: 0.0594 - val_category_output_acc: 0.6038 - val_category_output_precision: 0.7961 - val_category_output_recall: 0.4927 - val_ingredients_output_precision: 0.7821 - val_ingredients_output_recall: 0.4149 - val_calorie_output_MAE: 0.5158 - val_carbs_output_MAE: 0.0675 - val_protein_output_MAE: 0

...

334/334 - 702s - loss: 1.6409 - category_output_loss: 1.0303 - ingredients_output_loss: 0.0177 - calorie_output_loss: 0.4624 - carbs_output_loss: 0.0607 - protein_output_loss: 0.0202 - fat_output_loss: 0.0495 - category_output_acc: 0.7112 - category_output_precision: 0.8592 - category_output_recall: 0.6021 - ingredients_output_precision: 0.8392 - ingredients_output_recall: 0.5346 - calorie_output_MAE: 0.4624 - carbs_output_MAE: 0.0607 - protein_output_MAE: 0.0202 - fat_output_MAE: 0.0495 - val_loss: 1.9653 - val_category_output_loss: 1.3498 - val_ingredients_output_loss: 0.0178 - val_calorie_output_loss: 0.4654 - val_carbs_output_loss: 0.0608 - val_protein_output_loss: 0.0196 - val_fat_output_loss: 0.0519 - val_category_output_acc: 0.6573 - val_category_output_precision: 0.7914 - val_category_output_recall: 0.5859 - val_ingredients_output_precision: 0.8336 - val_ingredients_output_recall: 0.5521 - val_calorie_output_MAE: 0.4654 - val_carbs_output_MAE: 0.0608 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer\model\assets


334/334 - 701s - loss: 1.5541 - category_output_loss: 0.9515 - ingredients_output_loss: 0.0173 - calorie_output_loss: 0.4565 - carbs_output_loss: 0.0597 - protein_output_loss: 0.0199 - fat_output_loss: 0.0492 - category_output_acc: 0.7321 - category_output_precision: 0.8666 - category_output_recall: 0.6293 - ingredients_output_precision: 0.8455 - ingredients_output_recall: 0.5478 - calorie_output_MAE: 0.4565 - carbs_output_MAE: 0.0597 - protein_output_MAE: 0.0199 - fat_output_MAE: 0.0492 - val_loss: 1.9609 - val_category_output_loss: 1.3478 - val_ingredients_output_loss: 0.0177 - val_calorie_output_loss: 0.4645 - val_carbs_output_loss: 0.0608 - val_protein_output_loss: 0.0194 - val_fat_output_loss: 0.0506 - val_category_output_acc: 0.6624 - val_category_output_precision: 0.7913 - val_category_output_recall: 0.5919 - val_ingredients_output_precision: 0.8360 - val_ingredients_output_recall: 0.5555 - val_calorie_output_MAE: 0.4645 - val_carbs_output_MAE: 0.0608 - val_protein_output_MAE: 0

INFO:tensorflow:Assets written to: ./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer\model\assets


334/334 - 711s - loss: 1.3569 - category_output_loss: 0.7732 - ingredients_output_loss: 0.0164 - calorie_output_loss: 0.4427 - carbs_output_loss: 0.0576 - protein_output_loss: 0.0193 - fat_output_loss: 0.0477 - category_output_acc: 0.7779 - category_output_precision: 0.8883 - category_output_recall: 0.6900 - ingredients_output_precision: 0.8611 - ingredients_output_recall: 0.5739 - calorie_output_MAE: 0.4427 - carbs_output_MAE: 0.0576 - protein_output_MAE: 0.0193 - fat_output_MAE: 0.0477 - val_loss: 1.9531 - val_category_output_loss: 1.3520 - val_ingredients_output_loss: 0.0176 - val_calorie_output_loss: 0.4597 - val_carbs_output_loss: 0.0563 - val_protein_output_loss: 0.0193 - val_fat_output_loss: 0.0483 - val_category_output_acc: 0.6655 - val_category_output_precision: 0.7837 - val_category_output_recall: 0.6063 - val_ingredients_output_precision: 0.8335 - val_ingredients_output_recall: 0.5663 - val_calorie_output_MAE: 0.4597 - val_carbs_output_MAE: 0.0563 - val_protein_output_MAE: 0

#### Fine tune the convolution base without weights transfer

This code sets up the FlatEfficientNetB1Model instance and loads the model previously trained from scratch without pretrained weights. It then unfreezes part of the convolution base starting from the 117th layer and compiles the model with a reduced learning rate for fine-tuning. After printing the model summary, it begins training on the comprehensive dataset for an additional 10 epochs. This process aims to further optimize the model's performance by fine-tuning the convolutional layers after the initial training phase.

In [22]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)
flat_efficientnet.load_model(
    "./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/model"
)
flat_efficientnet.unfreeze_convolution_base(117)
flat_efficientnet.compile_model(keras.optimizers.Adam(learning_rate=1e-5))
flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 augmentation_layers (Functiona  (None, 224, 224, 3)  0          ['input_12[0][0]']               Y          
 l)                                                                                                          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_13 (InputLayer)        [(None, 224, 224, 3  0           []                         

In [24]:
flat_efficientnet.train_model(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    verbose=2,
)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'flat_efficientnetB1/efficientnetb1/block2a_expand_bn/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.

### MobileNetv3

Here, we define the FlatMobileNetv3Model class which extends the FlatModel to utilize the MobileNetV3 architecture. This involves setting up preprocessing layers specific to MobileNetV3 and creating the convolutional block using MobileNetV3Large pretrained on ImageNet.

In [ ]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_mobilenetv3",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In this section, we train the FlatMobileNetv3Model on the main dataset (recipes5k, food101, nutrition5k) using a specified configuration. The training process will include the building, compiling, and training of the model.

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_mobilenetv3.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_mobilenetv3.compile_model(keras.optimizers.Adam())
flat_mobilenetv3.print_summary()

In [ ]:
flat_mobilenetv3.train_model(
    x=no_category_train, validation_data=no_category_val, epochs=2, verbose=1
)

### NASNet Mobile

In this section, we define the FlatNASNetModel class, which is derived from FlatModel. This model uses NASNetMobile as the convolutional base.

In [ ]:
class FlatNASNetModel(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_nasnet",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.nasnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        nasnet_convolution_layers = keras.applications.nasnet.NASNetMobile(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        nasnet_convolution_layers.trainable = False
        return nasnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_nasnet = FlatNASNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_nasnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_nasnet.compile_model(keras.optimizers.Adam())

flat_nasnet.print_summary()

In [ ]:
flat_nasnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=10, verbose=1
)

## Wide Slice Model

In [ ]:
class WideSliceModel(BaseModel):
    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        conv_layer = self.get_convolution_block()(prev_layer)
        wide_slice_layer = self.get_wideslice_conv_layers(prev_layer)(prev_layer)
        prev_layer = keras.layers.Concatenate()([conv_layer, wide_slice_layer])
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

    def get_wideslice_conv_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        rescale_layer = keras.layers.Rescaling(scale=1.0 / 255)(input_layer)
        wideslice_conv = keras.layers.Conv2D(filters=64, kernel_size=(5, 224))(
            rescale_layer
        )
        wideslice_conv = keras.layers.BatchNormalization()(wideslice_conv)
        wideslice_conv = keras.layers.ReLU()(wideslice_conv)
        wideslice_conv = keras.layers.GlobalMaxPool2D()(wideslice_conv)
        return keras.Model(
            inputs=input_layer, outputs=wideslice_conv, name="wideslice_conv"
        )

### EfficientNetB1

In [ ]:
class WideSliceEfficientNetModel(WideSliceModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "wideslice_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

wideslice_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
    independent_ingredients_units=[1024, 512, 256],
)
wideslice_efficientnet.compile_model(keras.optimizers.Adam())
wideslice_efficientnet.print_summary()

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset, epochs=5, verbose=2, validation_data=validation_dataset
)

#### Transfer weights from fine tuned flat model

The objective is to leverage the knowledge gained from fine-tuning the flat EfficientNetB1 model (fine_tuned_flat_efficientnet) and apply it to enhance the performance of the WideSliceEfficientNetModel (wideslice_efficientnet) by initializing its convolutional base with these learned weights and further training it on a combined dataset (train_dataset).

In [ ]:
fine_tuned_flat_efficientnet = keras.models.load_model(
    "./temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransferFineTuned/model"
)
fine_tuned_efficientnet_weights = fine_tuned_flat_efficientnet.get_layer(
    "efficientnetb1"
).get_weights()

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)

wideslice_efficientnet.load_model(
    "./temp/checkpoint/wideslice_efficientnetB1/finalCategoryNoTransfer/model"
)
wideslice_efficientnet.unfreeze_convolution_base(117)
wideslice_efficientnet.model.get_layer("efficientnetb1").set_weights(
    fine_tuned_efficientnet_weights
)
wideslice_efficientnet.compile_model(keras.optimizers.Adam(learning_rate=1e-5))
wideslice_efficientnet.print_summary()

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset,
    epochs=5,
    verbose=2,
    validation_data=validation_dataset,
)